In [1]:
import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
from pymatreader import read_mat
from scipy import sparse
import numpy as np
import os
from datetime import datetime, timedelta
import pandas
import cv2
import json
import pandas as pd
from amftrack.pipeline.paths.directory import *
from amftrack.util.sys import *
import pickle
from joblib import Parallel, delayed
from datetime import datetime
import matplotlib.pyplot as plt
from amftrack.pipeline.functions.post_processing.global_plate import *
from amftrack.pipeline.functions.post_processing.global_hypha import *
from amftrack.pipeline.functions.post_processing.time_hypha import *
from amftrack.pipeline.functions.post_processing.time_plate import *

/home/cbisot/pycode/MscThesis/amftrack/transfer/functions/transfer.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
directory = directory_project
# update_analysis_info(directory)
analysis_info = get_analysis_info(directory)

In [3]:
select = analysis_info.loc[analysis_info['Plate']!=22]
# select = analysis_info.loc[analysis_info['Plate']==94]

In [4]:
frames = []
for index, row in select.iterrows():
    folder = row['folder_analysis']
    path_time_plate_info = row['path_time_plate_info']
    plate = row['Plate']
    if os.path.isfile(f'{directory}{path_time_plate_info}'):
        time_plate_info = pd.read_json(f'{directory}{path_time_plate_info}',
       convert_dates=True).transpose()
        time_plate_info.index.name = 't'
        time_plate_info.reset_index(inplace=True)
        for indexo, rowt in time_plate_info.iterrows():
            t = rowt['t']
            path_hyph_info_t = f'{directory}{folder}/time_hypha_info/hyph_info_{t}.json'
            if os.path.isfile(path_hyph_info_t):
                hyph_info_t = pd.read_json(path_hyph_info_t,
       convert_dates=True).transpose()
                hyph_info_t['Plate']=rowt['Plate']
                hyph_info_t['folder']=folder
                hyph_info_t['t']=rowt['t']
                frames.append(hyph_info_t)
time_hypha_infos = pd.concat(frames)

In [6]:
path_save = f'/scratch-shared/amftrack/temp/time_hypha_infos.pick'
time_hypha_infos.to_pickle(path_save)
dir_drop = 'data_tables'
API = str(np.load(os.getenv('HOME')+'/pycode/API_drop.npy'))
upload(API,path_save,f'/{dir_drop}/time_hypha_infos.pick',chunk_size=256 * 1024 * 1024)
path_save = f'/scratch-shared/amftrack/temp/time_hypha_infos.csv'
time_hypha_infos.to_csv(path_save)
dir_drop = 'data_tables'
API = str(np.load(os.getenv('HOME')+'/pycode/API_drop.npy'))
upload(API,path_save,f'/{dir_drop}/time_hypha_infos.csv',chunk_size=256 * 1024 * 1024)

In [5]:
time_hypha_infos.loc[(time_hypha_infos['Plate']==48)]

,end,time_since_begin_exp,distance_final_pos,timedelta,time_since_emergence,speed,timestep,timestep_init,time_init,degree,...,in_ROI,Plate,folder,t,width_root_edge,tot_length_C,absolute_angle,density_window500,density_window1000,density_window2000
2,2,0,15.5242,4,0,0.43125,0,0,0,1,...,True,48,Analysis_1646051303910926503_0_51_Version11,0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0,19.4165,4,0,0,0,0,0,1,...,True,48,Analysis_1646051303910926503_0_51_Version11,0,NaN,NaN,NaN,NaN,NaN,NaN
6,6,0,23.0868,4,0,0,0,0,0,1,...,True,48,Analysis_1646051303910926503_0_51_Version11,0,NaN,NaN,NaN,NaN,NaN,NaN
7,7,0,50,4,0,0,0,0,0,1,...,True,48,Analysis_1646051303910926503_0_51_Version11,0,NaN,NaN,NaN,NaN,NaN,NaN
11,11,0,22987.7,4,0,0,0,0,0,1,...,True,48,Analysis_1646051303910926503_0_51_Version11,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122855,122855,196.367,133.417,4.03333,0,59.973,49,49,196.367,1,...,True,48,Analysis_1646051303910926503_0_51_Version11,49,NaN,NaN,NaN,NaN,NaN,NaN
123026,123026,196.367,139.628,4.03333,0,66.0546,49,49,196.367,1,...,False,48,Analysis_1646051303910926503_0_51_Version11,49,NaN,NaN,NaN,NaN,NaN,NaN
123129,123129,196.367,1080.97,4.03333,0,87.7597,49,49,196.367,1,...,False,48,Analysis_1646051303910926503_0_51_Version11,49,NaN,NaN,NaN,NaN,NaN,NaN
123197,123197,196.367,43.2782,4.03333,0,17.8852,49,49,196.367,1,...,False,48,Analysis_1646051303910926503_0_51_Version11,49,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
select = time_hypha_infos.loc[time_hypha_infos['Plate']==792]
hyphae = select['end'].unique()
select_hyphae = [hyph for hyph in hyphae if len(select.loc[select['end']==hyph])

In [12]:
hyphae

array([0, 1, 4, ..., 197833, 197843, 197853], dtype=object)